## Formel 1 Friksjon Analyse

### Læringsmål
- Bruke virkelige data til å beregne hastighet, akselerasjon og krumming (**Oppgave 1**)
- Bruke klassisk mekanikk til å estimere friksjonskoeffisienter (**Oppgave 2**)
- Bygg og forbedre en modell for fysikken som inngår (**Oppgave 2**, **Oppgave 3**)
- Sammenligne fysikkbaserte prediksjoner med ingeniørmessige realiteter (**Oppgave 2**, **Oppgave 3**, **Oppgave 4**)

## Introduksjon

Hva får en Formel 1 bil til å holde seg klistret til veien når den suser gjennom svingene i vanvittige hastigheter, hastigheter som ville sendt en vanlig bil rett av banen? Er det dekkene? Førernes ferdigheter? Eller kanskje noe helt annet?

I dette prosjektet skal vi dykke ned i Formel 1 racingens høyhastighetsverden. Ikke som tilskuere, men som dataforskere og fysikere. Ved hjelp av ekte telemetridata fra den britiske Grand Prix i 2025 skal vi analysere hvordan noen av verdens beste førere håndterer sving 15 på Silverstone – en av de raskeste og mest krevende svingene i sporten.

Du skal beregne hvor raskt bilene kjører, hvor skarpt de svinger, og hvor mye veigrep dekkene må generere for å unngå å skli av banen. Underveis bygger du en fysikkbasert modell for dekkgrep helt fra bunnen av, med klassisk mekanikk som utgangspunkt, og utvikler modellen trinn for trinn mot en mer komplett forståelse.

Dette prosjektet handler ikke bare om regning. Det handler om å forstå den grunnleggende fysikken som gjør det mulig for disse bilene å ta svinger i ekstreme hastigheter. Du starter med en enkel modell basert på Newtons lover og beregner hvilken friksjonskoeffisient som kreves for å komme trygt gjennom svingen. For å forstå hvordan disse bilene trosser grensene for vanlig kjøring, vil du også lære hvordan aerodynamikk fundamentalt endrer spillet. Ved å bygge en enkel modell for hvordan nedoverkraft (downforce) påvirker dekkgrep, vil du skape et mer presist og fysikkbasert bilde av hva som faktisk gjør det mulig for en F1-\ bil å takle slike hastigheter i svingene.

Dette er ikke bare en dataøvelse, men en reise gjennom modellbygging, vitenskapelig tenkning og anvendt fysikk – alt pakket inn i motorsportens rå spenning.

![British Grand Prix Circuit with Corners](Racing_track.png)

## Importere data
I den første delen av prosjektet skal vi bruke data fra Lando Norris sin bil under runde 49. Dette var den raskeste runden til den raskeste føreren under den britiske Grand Prix i 2025.

Dataene er lagret i en CSV-fil med navnet `NOR.csv`. Når filen lastes inn, vil du få en "dictionary" med to keys: `time` og `distance`. Key-en `time` inneholder en liste med tidsstempler for hvert datapunkt, og `distance` inneholder en liste med distanser som bilen har tilbakelagt ved hvert tidspunkt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("NOR.csv")

Norris_data = df.to_dict("list")
for key in Norris_data:
    Norris_data[key] = np.array(Norris_data[key])

## 1) Finne hastigheter
I denne delen skal du beregne hastigheten til Lando Norris sin bil gjennom hele runden. Dette er første steg i analysen av hvordan bilens fart varierer mellom ulike segmenter av banen.

Hovedfokuset vårt vil være sving 15, en høyhastighets høyresving. Som vist på bildet ovenfor, er denne svingen flankert av lange rettstrekninger på begge sider, noe som betyr at førerne går både inn og ut av svingen i svært høy fart. Det gjør den til et ideelt sted å studere dekkfriksjon, siden bilen må generere betydelig veigrep for å holde seg på banen uten å bremse særlig ned. Å forstå bilens hastighet gjennom denne delen er avgjørende for målet vårt om å beregne friksjonskoeffisienten som kreves for å ta svingen i høy hastighet.

### 1a) Sirkulær bevegelse

For å starte analysen vår, vil vi tilnærme sving 15 som en halvsirkel. Selv om dette er en forenkling, lar det oss bruke fysikken for sirkulær bevegelse for å bedre forstå hvordan bilen holder sin bane gjennom svingen.

Når et objekt beveger seg langs en sirkulær bane, opplever det sentripetalakselerasjon, som alltid peker mot sentrum av sirkelen. Formelen for denne akselerasjonen er:
$$ a_c = \frac{v^2}{r} $$

For å produsere denne innoverrettede akselerasjonen, må en netto kraft virke på objektet, rettet mot sentrum. Denne kraften kalles sentripetalkraften, og i tilfellet med en bil på en flat bane, kommer den utelukkende fra statisk friksjon mellom dekkene og veibanen:
$$ F_f = F_c = m a_c = m \frac{v^2}{r} $$

Med andre ord: friksjonen motarbeider ikke bilens bevegelse, men det er faktisk den som endrer retningen ved å trekke bilen innover mens den beveger seg fremover.

Tegn et fritt-legeme-diagram (free body diagram) av bilen i sving 15. Inkluder hastigheten $v$ og friksjonskraften $F_f$ i diagrammet ditt.

*Plasser diagrammet ditt her:*

### 1b) Hastighetsfunksjon
Siden dataene vi har kun inneholder distansen bilen har tilbakelagt og tilhørende tidspunkter, ønsker vi å utvide dataene til å også inkludere hastigheten til bilen ved hvert tidspunkt. Vi beregner hastigheten ved å bruke numerisk derivasjon på dataene. Hastigheten ved hvert punkt kan tilnærmes med formelen:
$$ v_i = \frac{d_i - d_{i-1}}{t_i - t_{i-1}} $$
hvor $v_i$ er hastigheten ved tid $t_i$, $d_i$ er distansen ved tid $t_i$, og $d_{i-1}$ er distansen ved forrige tidsskritt $t_{i-1}$.

Lag en funksjon `find_velocity(distances, times)` som tar inn arrayene med distanser og tidspunkter, og returnerer et nytt array med beregnede hastigheter for hvert tidssteg.

Bruk deretter denne funksjonen til å beregne hastigheten for føreren, og legg den til i den opprinnelige ordboken under en ny key kalt `"velocity"`.

*Merk: $v_0$ er ikke definert, så du må håndtere denne situasjonen på en passende måte.*

In [2]:
def find_velocity(distances: np.ndarray, times: np.ndarray) -> np.ndarray:
    """
    Velocities are calculated as the time derivative of the distance data.

    Parameters:
    -----------
    distances (np.ndarray): Array of distances covered by the racer at different time intervals.
    times (np.ndarray): Array of time intervals corresponding to the distance data.

    Returns:
    -----------
    velocities (np.ndarray): Array of calculated velocities for the racer.
    """
    
    # Din implementasjon her


    return ...

# Ekstrakter distanse og tid fra Norris_data
distance = Norris_data["distance"]  # m 
time = ... 

#Lag en ny key i Norris_data for hastighet
Norris_data["velocity"] = find_velocity(distance, time) 
print(Norris_data.keys()) #print alle keys i Norris_data for å se at "velocity" er lagt til

dict_keys(['time', 'distance', 'velocity'])


### 1c) Sving 15
Nå som vi har hastighetsdataene, kan vi fokusere på sving 15. Vi vil hente ut relevante datapunkter for sving 15 og analysere hastighetene på disse punktene.

I denne underseksjonen skal du plotte bilens hastighet gjennom hele sving 15. Dette vil hjelpe oss å visualisere hvordan bilens fart endres mens den kjører gjennom svingen.

In [ ]:
start_of_corner_15 = 4840#m 
end_of_corner_15 = 5150#m 


# Ekstrakter distanse og hastighet fra Norris_data
distances = ...#m
velocities = ...#m/s


start_idx = np.where(distances >= start_of_corner_15)[0][0]
end_idx = np.where(distances <= end_of_corner_15)[0][-1]

## Valgfritt å bruke hvis du vil se hastighetsprofilen før og etter svingen også
## Hvis du gjør det, sørg for å visualisere hvor svingen starter og slutter
into_corner_15 = np.where(distances >= 4500)[0][0] # Tilfeldig distanse før svingen (4500 meter inn i runden)
out_of_corner_15 = np.where(distances <= 5500)[0][-1] # Tilfeldig distanse etter svingen (5500 meter inn i runden)

# Slice distansene og hastighetene for sving 15
distance_slice = ...
velocity_slice = ...

plt.figure(figsize=(10, 6))

# Plot her 


plt.title(f"Velocity Profile for Lando Norris on Lap 49")
plt.xlabel("Distance (m)")
plt.ylabel("Velocity (m/s)")
plt.legend()
plt.grid()
plt.show()


### 1d) Gjennomsnittshastighet
Før vi går videre til dekkfriksjonen, ønsker vi å finne en tilnærmet gjennomsnittshastighet for sving 15.

Bruk indeksene `start_idx` og `end_idx` fra forrige steg til å beregne gjennomsnittshastigheten til Lando Norris sin bil i sving 15

In [ ]:
average_velocity = ...

print(f"Average velocity in corner 15: {average_velocity:.2f} m/s")

## 2) Dekkfriksjon
Nå som vi har bilens hastighet, kan vi snart starte oppdraget vårt med å finne friksjonskoeffisientene til dekkene.

For å gjøre dette, må vi forenkle situasjonen med noen rimelige antakelser:

1. Sving 15 er en perfekt halvsirkel. Dette betyr at bilen holder en konstant krumningsradius gjennom hele svingen, noe som lar oss bruke standardformlene for sentripetalakselerasjon og -kraft.

2. Føreren kjører med maksimal hastighet gjennom svingen, noe som betyr at bilen befinner seg på grensen av det dekkene klarer å gripe.

3. Banen er flat, altså uten dosering i svingen.

Med disse antakelsene kan vi gå videre og beregne friksjonskoeffisienten $μ$ ved å knytte friksjonskraften til den sentripetale kraften.

### 2a) Finne $μ$
I denne delen ønsker vi å finne en formel for friksjonskoeffisienten $μ$.

Den første antakelsen vi gjorde, lar oss bruke formelen for sentripetalakselerasjon $a_c = \frac{v^2}{r}$ for å finne friksjonskoeffisienten $μ$.
Siden føreren kjører med maksimal hastighet som dekkene kan håndtere uten å miste grep, kan vi si at den sentripetale kraften, $F_c = m a_c$, er lik friksjonskraften, $F_f = μ N$.

Finn et uttrykk for friksjonskoeffisienten $μ$, avhengig av gjennomsnittshastigheten, radiusen og massen.

*Løsning:*

### 2b) Lag en funksjon for å beregne friksjonskoeffisienten
Som du kan se fra formelen over, trenger vi å kjenne til hastigheten $v$ gjennom svingen, gravitasjonsakselerasjonen $g$, og radiusen $r$ til svingen for å kunne beregne friksjonskoeffisienten $μ$.

Vi har allerede gjennomsnittshastigheten tilgjengelig, og heldigvis for oss ble gravitasjonsakselerasjonen fastsatt av den tredje generalkonferansen for mål og vekt i 1901 til å være
$g = 9.81 , \text{m/s}^2$ [Standard gravity](https://en.wikipedia.org/wiki/Standard_gravity).

Så først må du finne radiusen til svingen. Bruk formelen for omkretsen av en sirkel, $C = 2 \pi r$, sammen med det faktum at sving 15 er en halvsirkel med lengde $L$, for å finne radiusen $r$ til svingen. Lengden er distansen mellom start- og sluttpunktet for sving 15, som ble gitt i forrige seksjon.

Lag deretter en funksjon `mu(v, r)` som tar inn gjennomsnittshastigheten gjennom svingen og radiusen til svingen, og returnerer friksjonskoeffisienten $μ$.

*Merk: Du kan bruke np.pi for verdien av π.*

In [ ]:
g = 9.81 # Gravitasjons akselerasjon i m/s^2
length_of_corner = ... # Lengden på svingen i meter
radius_of_corner = ... # Radiusen til svingen i meter

def mu(v: float, r: float) -> float:
    """Calculate the coefficient of friction required to maintain a given velocity in a circular path.
    
    Parameters:
    ----------
    v (float): Velocity in m/s.
    r (float): Radius of the circular path in meters.

    Returns:
    -------
    μ (float): Coefficient of friction required.

    """
    return ...



### 2c) Beregn friksjonskoeffisienten
Ved å bruke gjennomsnittshastigheten gjennom sving 15 og radiusen til svingen, beregn friksjonskoeffisienten $μ$.

In [ ]:
coefficient_of_friction = ...

print(f"Coefficient of friction for corner 15: {coefficient_of_friction:.2f}")

Du legger kanskje merke til at friksjonskoeffisienten er ganske høy. Vi forstår at F1-biler er konstruert for å ha ekstremt godt veigrep, men denne verdien virker urealistisk. La oss gå videre til neste seksjon for å finne ut hvorfor.

## 3) Forbedring av modellen
I forrige seksjon estimerte vi friksjonskoeffisienten for sving 15. Verdien vi kom frem til er imidlertid trolig urealistisk høy. Det forteller oss at modellen vår mangler noe viktig.

Det er én ting ved racerbiler vi ikke kan overse, og det er nedoverkraften (downforce). Kanskje det mest kjente kjennetegnet ved en F1-bil er den enorme bakvingen, som brukes til å skape nettopp nedoverkraft. Nedoverkraften er en kraft som presser bilen ned mot asfalten, øker dekkenes veigrep, og gjør det mulig for bilen å kjøre raskere gjennom svingene.

Med andre ord: bilen stoler ikke bare på sin egen vekt for å få grep, den bruker også aerodynamisk trykk.
I denne seksjonen skal vi utvide modellen vår til å ta hensyn til nedoverkraften, og undersøke hvordan dette påvirker den beregnede friksjonskoeffisienten. Dette vil gi oss et mer realistisk og fysikkbasert estimat på dekkgrep i høye hastigheter.

### 3a) Nedoverkraft (downforce)
Tidligere antok vi at friksjonskraften alene stod for å gi den sentripetale kraften som trengs for å holde bilen i en sirkulær bane. Men racerbiler, spesielt F1-biler, genererer betydelig nedoverkraft på grunn av sin aerodynamiske utforming. Denne kraften øker normalkraften, og dermed øker også den maksimale friksjonskraften dekkene kan produsere uten å miste grep.

Dermed er ikke normalkraften $N$ lenger bare bilens vekt ($mg$), men inkluderer også nedoverkraften $F_d$:
$$ N = mg + F_d $$

hvor $F_D$ er den aerodynamiske nedoverkraften. En forenklet modell for nedoverkraften er:
$$F_d = C_D v²$$

hvor $C_D$ er en effektiv nedoverkraftkoeffisient, og $v$ er hastigheten.

Med denne nye definisjonen av normalkraften, finn et nytt uttrykk for friksjonskoeffisienten $μ$ som inkluderer nedoverkraften.


*Løsning:*

### 3b) Lag en funksjon for å beregne den nye friksjonskoeffisienten

Nå er det på tide å lage en forbedret funksjon for friksjonskoeffisienten som tar hensyn til nedoverkraft.

La oss redefinere funksjonen `mu(v, r, C_D, m)` slik at den inkluderer nedoverkraftleddet. Den nye funksjonen skal ta inn nedoverkraftkoeffisienten $C_D$, bilens masse $m$, radiusen til svingen $r$, og gjennomsnittshastigheten $v$.

Bilens masse, inkludert føreren, er omtrent 800 kg, og nedoverkraftkoeffisienten $C_D$ er omtrent 3.2, selv om den kan variere mellom ulike biler.


In [ ]:
def mu(v: float, r: float, C_D: float = 3.2, m: float = 800) -> float:
    """Calculate the coefficient of friction required to maintain a given velocity in a circular path.
    
    Parameters:
    ----------
    v (float): Velocity in m/s.
    r (float): Radius of the circular path in meters.
    C_D (float): Drag coefficient (default is 3.2).
    m (float): Mass of the car in kg (default is 800).
    

    Returns:
    -------
    μ (float): Coefficient of friction required.
    """
    return ...

### 3c) Beregn den nye friksjonskoeffisienten
Nå som vi har oppdatert mu-funksjonen til å inkludere nedoverkraft, kan du bruke den og se om den nye friksjonskoeffisienten er mer realistisk.

In [ ]:
coefficient_of_friction = ...

print(f"Coefficient of friction for corner 15: {coefficient_of_friction:.2f}")

Sannsynligvis vil du ha funnet ut at den nye friksjonskoeffisienten er mye lavere enn den forrige. Dette skyldes at nedoverkraften øker dekkens grep, noe som gjør at bilen kan kjøre raskere gjennom svingene uten å skli.

Prøv å søke på nettet etter gjennomsnittlig friksjonskoeffisient for F1-dekk, og se om resultatet ditt ligger i samme størrelsesorden.

## 4) Friksjonskoeffisient for alle førerne
Nå som vi har en funksjon for å beregne friksjonskoeffisienten, kan vi bruke den på alle førerne som kvalifiserte seg til finaleløpene i British Grand Prix 2025.

Nedenfor har vi importert dataene for alle førerne. Dette datasettet er en nestet ordbok ("nested dictionary"), der hver nøkkel er navnet på en fører, og for hver fører finnes det en ordbok med nøklene `distance` og `time`, som inneholder tilbakelagt distanse og tid ved hvert tidspunkt, henholdsvis.

In [ ]:
driver_codes = ['VER', 'TSU', 'HUL', 'PIA', 'NOR', 'SAI', 'OCO', 'ALO', 'LEC', 'BEA', 'HAM', 'ALB', 'STR', 'RUS', 'GAS']
Racing_data = {}

for code in driver_codes:
    df = pd.read_csv(f"Racing_data/{code}.csv") # Les inn data for hver fører
    Racing_data[code] = { #Lag en ordbok for hver fører
        "time": df["time"].to_numpy(),
        "distance": df["distance"].to_numpy()
    }

print(Racing_data.keys()) # hver nøkkel er en førerkode, og verdien er en ordbok med tid- og distanse-arrays

For de av oss som ikke er kjent med forkortelsene for førernavnene, her er en tabell med fullstendige navn på førerne:

|Fullt navn | Forkortelse |
|------------|-------|
|Lando Norris| NOR   |
|Lance Stroll| STR   |
|Nico Hulkenberg| HUL   |
|George Russell| RUS   |
|Charles Leclerc| LEC   |
|Oliver Bearman| BEA   |
|Oscar Piastri| PIA   |
|Max Verstappen| VER   |
|Lewis Hamilton| HAM   |
|Pierre Gasly| GAS   |
|Fernando Alonso| ALO   |
|Carlos Sainz| SAI   |
|Alex Albon| ALB   |
|Esteban Ocon| OCO   |

### 4a) Fnn alle hastigheter
Nå at vi har dataene, kan vi bruke `find_velocity(data)`-funksjonen vi laget tidligere for å finne hastighetene for alle førerne.


In [ ]:

for racer_key in Racing_data.keys():
    distance = ...
    time = ...
    Racing_data[racer_key]["velocity"] = ...


### 4b) Finn gjennomsnittshastighetene

I denne seksjonen vil vi beregne gjennomsnittshastighetene for hver fører gjennom hele sving 15. Hvilken racer var den raskeste gjennom sving 15?


In [ ]:
for racer_key in Racing_data.keys():
    distances = ...
    velocities = ...
    
    start_idx = np.where(distances >= start_of_corner_15)[0][0]
    end_idx = np.where(distances <= end_of_corner_15)[0][-1]
    
    average_velocity = ...
    
    print(f"Average velocity in corner 15 for {racer_key}: {average_velocity:.2f} m/s")

    ### Lagre en ny key i Racing_data for gjennomsnittshastighet
    ### Hint: Du kan lage en ny nøkkel ved å: Racing_data[racer_key]["new_key"] = new_data


*Løsning:*

### 4c) Beregn friksjonskoeffisientene
Nå har vi funnet gjennomsnittshastighetene for alle førerne for å se hvem som var raskest gjennom sving 15.
Det neste vi vil gjøre, er å beregne friksjonskoeffisientene for alle førerne, ved å bruke mu-funksjonen vi laget, med gjennomsnittshastighetene vi nettopp fant.

Finn friksjonskoeffisientene for alle førerne, samt gjennomsnittet og standardavviket for friksjonskoeffisientene mellom alle førerne.

*Tips: Du kan bruke np.mean() og np.std() fra NumPy-biblioteket for å beregne gjennomsnitt og standardavvik av friksjonskoeffisientene.*

In [ ]:
mus = []
for racer_key in Racing_data.keys():
    #ekstrakt data fra forrige steg
    
    #Finn friksjonskoeffisienten ved å bruke mu-funksjonen 

    # Lagre friksjonskoeffisienten i listen

    print(f"Coefficient of friction for {racer_key}: {coefficient_of_friction:.2f}")

mean_mu = ...
std_mu = ...

print(f"Average coefficient of friction for all racers: {mean_mu:.2f} ± {std_mu:.2f}")